In [ ]:
#%debug
%load_ext autoreload
%autoreload 2

from os.path import join as pjoin

import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
#%run -i ../exec_HPC.py 0
from config2 import path_code
from IPython import get_ipython; ipython = get_ipython()

locs = {}

from config2 import subjects

import numpy as np
from os.path import join as pjoin
dir_rr = '/home/demitau/ownCloud/Current/merr_data/Romain_calc_res/'
an = 'prevfeedback_preverrors_errors_prevbelief'
random = pjoin(dir_rr, f'sub01_WGPOZPEE_randompartial_scores_{an}_broad.npy')
stable = pjoin(dir_rr, f'sub01_WGPOZPEE_stablepartial_scores_{an}_broad.npy')
scores_romain_b2b = {}

scores_romain_b2b['stable'] = np.load(stable)# 5 splits each
scores_romain_b2b['random'] = np.load(random)

display(scores_romain_b2b)
print(an.split('_'))
# time_lock = target,  control = feedback    ( I had control = movement)

In [ ]:
for subject in subjects[:1]:
#for subject in subjects[1:]:
    print('   ', subject)
    n_jobs = 10
    hpass = 'no_filter'
    regression_type = 'Ridge'
    ipython.run_line_magic('run',
        f' -i {path_code}/spoc_home_with_prev_error2_orig.py')

In [ ]:
targets = np.array(behav_df['target_inds'])
targets_stable = targets[stable]
targets_random = targets[random]
# Feedback positions
feedback = np.array(behav_df['feedback'])
feedback_stable = feedback[stable]
feedback_random = feedback[random]
feedbackX = np.array(behav_df['feedbackX'])
feedbackX_stable = feedbackX[stable]
feedbackX_random = feedbackX[random]
feedbackY = np.array(behav_df['feedbackY'])
feedbackY_stable = feedbackY[stable]
feedbackY_random = feedbackY[random]
# Movement positions
movement = np.array(behav_df['org_feedback'])
movement_stable = movement[stable]
movement_random = movement[random]
# Error positions
errors = np.array(behav_df['error'])
errors_stable = errors[stable]
errors_random = errors[random]
abs_errors_stable = np.abs(errors_stable)
abs_errors_random = np.abs(errors_random)
# Belief
belief = np.array(behav_df['belief'])
belief_stable = belief[stable]
belief_random = belief[random]
# keep only non_hit trials
non_hit_stable = point_in_circle(targets_stable, target_coords,
                                 feedbackX_stable, feedbackY_stable,
                                 radius_target + radius_cursor)
non_hit_random = point_in_circle(targets_random, target_coords,
                                 feedbackX_random, feedbackY_random,
                                 radius_target + radius_cursor)

In [ ]:
DEBUG

In [ ]:
partial_scores_many = []

In [ ]:
times = epochs.times
if control == 'feedback':
    analysis_name = 'prevfeedback_preverrors_errors_prevbelief'
elif control == 'movement':
    analysis_name = 'prevmovement_preverrors_errors_prevbelief'

# Regression for classic decoding
alphas = np.logspace(-5, 5, 12)
est = make_pipeline(SPoC(n_components=5, log=True, reg='oas',
                         rank='full'),
                    RidgeCV(alphas=alphas))
# Regressions for the B2B
G = make_pipeline(SPoC(n_components=5, log=True, reg='oas',
                       rank='full'),
                  RidgeCV(alphas=alphas, fit_intercept=False))
H = LinearRegression(fit_intercept=False)
if DEBUG:
    b2b = B2B_SPoC(G=G, H=H, n_splits=60)
else:
    #b2b = B2B_SPoC(G=G, H=H, n_splits=30)

    ## NEW
    n_splits = 200
    b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type='across_splits',
               n_jobs=10)


# Cross-validation
cv = KFold(nb_fold, shuffle=True)

#  Run decoding
envs = ['stable', 'stable', 'stable', 'stable']
#envs = ['stable', 'random']
if DEBUG:
    if isinstance(env_to_run,list):
        envs = env_to_run
    else:
        envs = [env_to_run]
#for env in ['stable', 'random']:
#for env in ['stable', 'random'][::-1]:
for env in envs:
    if env == 'stable':
        ep = epochs['20', '21', '22', '23', '30']
        prev_errors_stable = np.insert(errors_stable, 0, 0)[:-1]
        prev_targets_stable = np.insert(targets_stable, 0, 0)[:-1]
        prev_movement_stable = np.insert(movement_stable, 0, 0)[:-1]
        prev_feedback_stable = np.insert(feedback_stable, 0, 0)[:-1]
        prev_belief_stable = np.insert(belief_stable, 0, 0)[:-1]
        if control == 'feedback':
            analysis_value = [prev_feedback_stable, prev_errors_stable,
                              errors_stable, prev_belief_stable]
        elif control == 'movement':
            analysis_value = [prev_movement_stable, prev_errors_stable,
                              errors_stable, prev_belief_stable]
        non_hit = non_hit_stable
        non_hit = np.array(non_hit)
        # remove trials following hit (because no previous error)
        non_hit = ~(~non_hit | ~np.insert(non_hit, 0, 1)[:-1])
        non_hit[[0, 192]] = False  # Removing first trial of each block
    elif env == 'random':
        ep = epochs['25', '26', '27', '28', '35']
        prev_errors_random = np.insert(errors_random, 0, 0)[:-1]
        prev_targets_random = np.insert(targets_random, 0, 0)[:-1]
        prev_movement_random = np.insert(movement_random, 0, 0)[:-1]
        prev_feedback_random = np.insert(feedback_random, 0, 0)[:-1]
        prev_belief_random = np.insert(belief_random, 0, 0)[:-1]
        if control == 'feedback':
            analysis_value = [prev_feedback_random, prev_errors_random,
                              errors_random, prev_belief_random]
        elif control == 'movement':
            analysis_value = [prev_movement_random, prev_errors_random,
                              errors_random, prev_belief_random]
        non_hit = non_hit_random
        non_hit = np.array(non_hit)
        # remove trials following hit (because no previous error)
        non_hit = ~(~non_hit | ~np.insert(non_hit, 0, 1)[:-1])
        non_hit[[0, 192]] = False  # Removing first trial of each block
    X = ep.pick_types(meg=True, ref_meg=False)._data
    #wh = (times > -0.45) & (times < -0.05)
    # NEW
    wh = (times > -0.45) & (times < 0.05)
    X = X[non_hit]
    X = X[:, :, wh]
    Y = np.array(analysis_value)
    Y = Y[:, non_hit].T

    ######################  DEBUG to check var values
    if DEBUG:
        from config2 import DEBUG_ntrials,DEBUG_nchannels
        X = X[:DEBUG_ntrials,:DEBUG_nchannels]
        Y = Y[:DEBUG_ntrials]
    ######################  DEBUG to check var values
        #sys.exit(0)

    # Classic decoding
    dd = 0
    if dd:
        scoring = make_scorer(scorer_spearman)
        scores = list()
        for ii in range(Y.shape[1]):
            y = Y[:, ii]
            with mne.use_log_level('warning'):
                score = cross_val_multiscore(est, X, y=y, cv=cv, scoring=scoring,
                                            n_jobs=n_jobs)
            score = score.mean(axis=0)
            scores.append(score)
        scores = np.array(scores)

        print(env, scores)
    
    ######################  DEBUG to check var values
    #sys.exit(0)


    # run partial decoding
    Y = scale(Y)
    with mne.use_log_level('warning'):
        b2b.fit(X, Y)
    partial_scores = np.diag(b2b.E_)
    
    partial_scores_many += [partial_scores]
    
    print(n_splits, env, partial_scores)
    # save scores
    fname = op.join(results_folder,
                    '%s_%sscores_%s_%s.npy' % (subject, env,
                                               analysis_name, freq_name))
    np.save(fname, np.array(scores))
    fname = op.join(results_folder,
                    '%s_%spartial_scores_%s_%s.npy' % (subject, env,
                                                       analysis_name,
                                                       freq_name))
    np.save(fname, np.array(partial_scores))
    
    print('Finished')

In [ ]:
ps = ICA, hpass, freq_name,  control, time_locked,   env
pss = ';'.join( list( map(str,ps) ) )
print(pss)
fnf = op.join(results_folder, f'{pss}_XY.npz'); 
print(fnf)
np.savez_compressed(fnf, X=X,Y=Y)

In [ ]:
partial_scores

In [ ]:
print(fname)

In [ ]:
subject

In [ ]:
from datetime import datetime
#prep = f'/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/results/spoc_home2_{hpass}_almost_orig/'
prep = f'/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/results/spoc_home2_{hpass}_orig/'
for env in ['stable','random']:
    p = prep + f'{subject}_{env}partial_scores_prevmovement_preverrors_errors_prevbelief_broad.npy'
    dt = datetime.fromtimestamp( os.stat(p).st_mtime )
    r = np.load(p)
    #print(env, r.mean(), r.std() )
    print( env, list( map( lambda x: f'{x:.5f}', r) ), dt )


In [ ]:
stable ['-0.00514', '0.03931', '0.03942', '0.05747'] 2023-04-18 12:23:00.949813
random ['0.00009', '-0.00360', '-0.01051', '0.00283'] 2023-04-18 12:24:57.069810

In [ ]:
X,y

# compare spoc

In [ ]:
env = 'stable'

In [ ]:
from base2_orig import B2B_SPoC as B2B_SPoC_old
from base2 import  B2B_SPoC

from mne.decoding import (cross_val_multiscore, LinearModel,
                          SPoC)

G = make_pipeline(SPoC(n_components=5, log=True, reg='oas',
                       rank='full'),
                  RidgeCV(alphas=alphas, fit_intercept=False))
H = LinearRegression(fit_intercept=False)

n_splits = 20
ensemble = ShuffleSplit(n_splits=n_splits, 
        test_size=.5, random_state = 0)

# TODO many runs with diff random states to check
print(env)

b2b_type2b2b = {}

pt = 'across_splits'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=10, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

pt = 'across_splits_and_dims'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=10, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

pt = 'no'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=10, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

# b2b = B2B_SPoC_old(G=G, H=H, n_splits=n_splits,
#                    ensemble = ensemble)
# b2b_type2b2b[f'old'] = b2b

b2b_type2res = {}
for b2b_type, b2b  in b2b_type2b2b.items():
    with mne.use_log_level('warning'):
        b2b.fit(X, Y)
    partial_scores = np.diag(b2b.E_)
    print(b2b_type, partial_scores)
    b2b_type2res[b2b_type] = partial_scores
    


In [ ]:
b2b_type2b2b.keys()

# debug

In [ ]:
import numpy as np
fnf = '/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/results/spoc_home2_no_filter_orig/with_ICA;no_filter;broad;movement;target;stable_XY.npz'
f = np.load(fnf)

import numpy as np
from sklearn.model_selection import ShuffleSplit

X = f['X']
Y = f['Y']
print( X.shape)  # trial x sensors x time 
print( Y.shape)  # trial x  dim

In [ ]:


Y_test = np.array([[-0.35002536],
       [-1.48563779],
       [ 1.24180457],
       [-1.13094076]])

X_test = np.array([[[-2.11474666e-14,  6.48929128e-15]],

       [[-7.59599106e-14, -1.82705380e-14]],

       [[ 9.82457510e-14,  6.47432584e-14]],

       [[ 2.54159085e-14,  3.92063100e-14]]])

n_splits = 2
ensemble = ShuffleSplit(n_splits=n_splits, 
        test_size=.5, random_state = 0)
list(ensemble.split(X_test,Y_test))
print('s')

In [ ]:
nt = 6  # cannot have less
X_test = X[:nt, :2, :2]
Y_test = Y[:nt, :2]

print( X_test.shape)
print( Y_test.shape)

n_splits = 3
ensemble = ShuffleSplit(n_splits=n_splits, 
        test_size=.5, random_state = 0)
list(ensemble.split(X,Y))
print('s')

In [ ]:
# import importlib as il
# import base2
# il.reload(base2)

%load_ext autoreload
%autoreload 2
from os.path import join as pjoin

import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))

from mne.decoding import (cross_val_multiscore, LinearModel,
                          SPoC)
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression

import mne
alphas = np.logspace(-5, 5, 12)

G = make_pipeline(SPoC(n_components=5, log=True, reg='oas',
                       rank='full'),
                  RidgeCV(alphas=alphas, fit_intercept=False))
H = LinearRegression(fit_intercept=False)

from base2_orig import B2B_SPoC as B2B_SPoC_old
from base2 import  B2B_SPoC

n_jobs_test = 2


b2b_type2b2b = {}

pt = 'across_splits_and_dims'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=n_jobs_test, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

pt = 'across_splits'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=n_jobs_test, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

# b2b = B2B_SPoC_old(G=G, H=H, n_splits=n_splits,
#                    ensemble = ensemble)
# b2b_type2b2b[f'old'] = b2b

# H_sets are the same

b2b_type2res = {}
for b2b_type, b2b  in b2b_type2b2b.items():
    with mne.use_log_level('warning'):
        b2b.fit(X_test, Y_test)
    partial_scores = np.diag(b2b.E_)
    print('--- res ', b2b_type, partial_scores)
    b2b_type2res[b2b_type] = partial_scores

d = b2b_type2res['parallel_across_splits'] - b2b_type2res['parallel_across_splits_and_dims']
print('diff = ', np.max(np.abs(d)))

In [ ]:
np.__version__

In [ ]:
mne.__version__

In [ ]:
import mne


In [ ]:
X.shape

In [ ]:
n_jobs_test = 10

nt = 150  
ndim = Y.shape[1]
nsens = 150
ntimes = 80
n_splits = 10

# nt = 1500  
# ndim = Y.shape[1]
# nsens = 1500
# ntimes = 8000
#n_splits = 30

# nt = 15
# ndim = Y.shape[1] 
# nsens = 30
# ntimes = 20
X_test = X[:nt, :nsens, :ntimes]
Y_test = Y[:nt, :ndim]
#X_test = X[:nt, :1, :2]  -- works
#Y_test = Y[:nt, :1]


print( X_test.shape)
print( Y_test.shape)


ensemble = ShuffleSplit(n_splits=n_splits, 
        test_size=.5, random_state = 0)
list(ensemble.split(X,Y))

b2b_type2b2b = {}

pt = 'across_splits'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=n_jobs_test, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

pt = 'across_splits_and_dims'
b2b = B2B_SPoC(G=G, H=H, n_splits=n_splits, parallel_type=pt,
           n_jobs=n_jobs_test, ensemble = ensemble)
b2b_type2b2b[f'parallel_{pt}'] = b2b

b2b_as = b2b_type2b2b['parallel_across_splits']
b2b_asd = b2b_type2b2b['parallel_across_splits_and_dims']

In [ ]:
import time
b2b_type2res = {}
for b2b_type, b2b  in b2b_type2b2b.items():
    print('s', time.time())
    with mne.use_log_level('warning'):
        b2b.fit(X_test, Y_test)
    partial_scores = np.diag(b2b.E_)
    print('--- res ', b2b_type, partial_scores)
    print('--------------------------------')
    print('--------------------------------')
    b2b_type2res[b2b_type] = partial_scores
    
    print('e', time.time())
    
d = b2b_type2res['parallel_across_splits'] - b2b_type2res['parallel_across_splits_and_dims']
print('diff = ', np.max(np.abs(d)))

In [ ]:
%%timeit
b2b_asd.fit(X_test, Y_test)

In [ ]:
%%timeit
b2b_as.fit(X_test, Y_test)

In [ ]:
import gc; gc.collect()

In [ ]:
import sys; sys.executable

In [ ]:
from sklearn.base import _pprint

In [ ]:
b2b_type2res

In [ ]:
Y_test.shape

In [ ]:
# Y hat for only split has shape 4,2 for    otehr -- 4

In [ ]:
for b2b_type, b2b  in b2b_type2b2b.items():
    H_hats = b2b.H_hats
    print(b2b_type, H_hats.shape)
    display(H_hats)
    display(H_hats.reshape((3,2,2)))

In [ ]:
X_test.shape

In [ ]:
from base2 import _b2b_fit_one_split
_b2b_fit_one_split(0,X_test,Y_test, [0], [1], G,H)